In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Conv1D,MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
import matplotlib.pyplot as plt

In [4]:
tf.__version__

'2.10.0'

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/val'

In [6]:
from glob import glob

In [7]:
folders = glob('Dataset/*')

In [8]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 shuffle=False,
                                                 class_mode = 'categorical')

Found 1396 images belonging to 9 classes.


In [10]:
test_set = test_datagen.flow_from_directory('Dataset/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'categorical')

Found 349 images belonging to 9 classes.


In [11]:
model = Sequential()

model.add(Conv2D(filters=32,padding="same",kernel_size=3, activation='LeakyReLU', strides=2, input_shape=[224, 224, 3]))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(64,kernel_size = (1,1),strides = (1,1),activation='LeakyReLU'))
model.add(Conv2D(192,kernel_size = (3,3),strides = (1,1),activation='LeakyReLU'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())

model.add(Dense(4096,activation = 'LeakyReLU'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation = 'LeakyReLU'))
model.add(Dropout(0.5))

model.add(Dense(9,activation = 'Softmax'))


opt = SGD(learning_rate=0.01, momentum=0.9) #stochastic Gradient Descent
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])



In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 37, 37, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 64)        2112      
                                                                 
 conv2d_2 (Conv2D)           (None, 35, 35, 192)       110784    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 192)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 23232)             0

In [ ]:
# Training the CNN on the Training set and evaluating it on the Test set
r=model.fit(x = training_set, validation_data = test_set, epochs = 50)

Epoch 1/50
44/44 [==============================] - 54s 1s/step - loss: 2.3421 - accuracy: 0.0301 - val_loss: 2.1617 - val_accuracy: 0.1375
Epoch 2/50
44/44 [==============================] - 54s 1s/step - loss: 2.3577 - accuracy: 0.1375 - val_loss: 2.1588 - val_accuracy: 0.1805
Epoch 3/50
44/44 [==============================] - 58s 1s/step - loss: 2.3688 - accuracy: 0.0501 - val_loss: 2.0671 - val_accuracy: 0.3123
Epoch 4/50
44/44 [==============================] - 57s 1s/step - loss: 2.1044 - accuracy: 0.2185 - val_loss: 1.9697 - val_accuracy: 0.2407
Epoch 5/50
44/44 [==============================] - 57s 1s/step - loss: 2.1476 - accuracy: 0.2070 - val_loss: 1.6623 - val_accuracy: 0.3438
Epoch 6/50
44/44 [==============================] - 58s 1s/step - loss: 1.8651 - accuracy: 0.2815 - val_loss: 1.5062 - val_accuracy: 0.3410
Epoch 7/50
44/44 [==============================] - 59s 1s/step - loss: 1.7170 - accuracy: 0.3087 - val_loss: 1.6232 - val_accuracy: 0.3009
Epoch 8/50
44/44 [==

In [1]:
import numpy as np
batch_size = 32
Y_pred = model.predict_generator(test_set, steps = np.ceil(test_set.samples / test_set.batch_size), verbose=1, workers=0)
y_pred = np.argmax(Y_pred, axis=1)

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix
#confusion_matrix(y_true, y_test_pred)
confusion_matrix(test_set.classes, y_pred)

In [ ]:
Y_pred = model.predict_generator(test_set)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(test_set.classes, y_pred)

In [ ]:
cm

In [ ]:
Y_pred1 = model.predict_generator(training_set)
y_pred1 = np.argmax(Y_pred1, axis=1)
print('Confusion Matrix')
cm1 = confusion_matrix(training_set.classes, y_pred1)

In [ ]:
cm1

In [ ]:
cmm = cm1 + cm
cmm

In [ ]:
model.save('my_model.h5')

In [ ]:
#loading the model as new model
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

In [ ]:
f,ax=plt.subplots(2,1) #Creates 2 subplots under 1 column

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(new_model.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(new_model.history.history['val_loss'],color='r',label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(new_model.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(new_model.history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()